In [ ]:
#Run every cell, in order, until instructed otherwise

In [1]:
import pandas as pd
import numpy as npy
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
model=LogisticRegression()
import itertools

In [2]:
def subsets(n):
    lists=[]
    for i in range(1,n+1):
        lists.append(i)
    subsets=[[]]
    n=len(lists)
    for i in range(1,n+1):
        data = itertools.combinations(lists,i)
        subsets.extend(list(data))
    return subsets

In [3]:
ufc_fights= pd.read_csv('ufc_fights.csv')
ufc_fights_crap= pd.read_csv('ufc_fights_crap.csv')
ufcfightscrap= pd.read_csv('ufc_fights_crap.csv')
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')

/usr/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (36,37,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#adding mask to get rid of unknown stance
stance_mask_a=ufc_fights['fighter_stance']!=5
stance_mask_b=ufc_fights['opponent_stance']!=5

#gets rid of all unknown, NaN in ufc_fights_stance
mask=ufc_fights['fighter_wins'].astype(str) != 'unknown'
for col in ufc_fights.columns:
    mask1=ufc_fights[col].astype(str) != 'unknown'
    mask2=ufc_fights[col].astype(str) != 'NaN'
    mask3=ufc_fights[col].astype(str) != '--'
    mask=mask&mask1&mask2&mask3

#gets rid of ufc debut in ufc_fights_stance
maska=ufc_fights['fighter_inf_sig_strikes_attempts_avg'] != 0
maskb=ufc_fights['opponent_inf_sig_strikes_attempts_avg'] != 0

#Getting rid of all fights whose method is DQ, Overturned, or CNC or any draws
method_mask1=ufc_fights['method']!='DQ'
method_mask2=ufc_fights['method']!='Overturned'
method_mask3=ufc_fights['method']!='CNC'
method_mask4=ufc_fights['result']!='D'
method_mask=method_mask1&method_mask2&method_mask3&method_mask4

#forming total mask
mask=mask&maska&maskb&stance_mask_a&stance_mask_b&method_mask

#applies all masks resulting in ufc_fights_cleaned (annoying notation... clean the whole thing up)
ufc_fights_cleaned=ufc_fights[mask]

#Now lets get rid of the different types of decisions (all become DEC)
ufc_fights_cleaned['method'] = ufc_fights_cleaned['method'].replace(['U-DEC'],'DEC')
ufc_fights_cleaned['method'] = ufc_fights_cleaned['method'].replace(['S-DEC'],'DEC')
ufc_fights_cleaned['method'] = ufc_fights_cleaned['method'].replace(['M-DEC'],'DEC')

#change all columns to have float values
for col in ufc_fights_cleaned.columns:
    try:
        ufc_fights_cleaned[col]=ufc_fights_cleaned[col].astype(float)
    except:
        None

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/p

In [5]:
record_statistics=[u'fighter_wins', 
                   u'fighter_losses',
                   u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                    u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_wins', 
                   u'opponent_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', 
                   u'opponent_ko_losses', 
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', 
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',
                u'fighter_height',
                    u'fighter_reach',  
                u'opponent_age',  
                u'opponent_height',
                    u'opponent_reach']

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']

#forming a dataframe where columns are differences between fighter and opponent fight statistics
mrs=len(record_statistics)/2
record_diff_2_df=pd.DataFrame()
for i in range(mrs):
    record_diff_2_df[record_statistics[i]+'_diff_2']=ufc_fights_cleaned[record_statistics[i]]-ufc_fights_cleaned[record_statistics[i+mrs]]
    
#Lets start building a dataframe of all the stats which we know perform best
ufc_best_df=pd.DataFrame()
ufc_best_df['fighter']=ufc_fights_cleaned['fighter']
ufc_best_df['opponent']=ufc_fights_cleaned['opponent']
ufc_best_df['result']=ufc_fights_cleaned['result']
ufc_best_df['method']=ufc_fights_cleaned['method']
ufc_best_df['date']=ufc_fights_cleaned['date']
for col in record_diff_2_df.columns:
    ufc_best_df[col]=record_diff_2_df[col]

#moving on to physical_stats:
physical_stats_diff_df=pd.DataFrame()
physical=[ [u'fighter_age',u'opponent_age'],[u'fighter_height',u'opponent_height'],[u'fighter_reach',u'opponent_reach']]
physical_stats_diff_df['age_diff']=ufc_fights_cleaned['fighter_age'].apply(float)-ufc_fights_cleaned['opponent_age'].apply(float)
physical_stats_diff_df['height_diff']=ufc_fights_cleaned['fighter_height'].apply(float)-ufc_fights_cleaned['opponent_height'].apply(float)
physical_stats_diff_df['reach_diff']=ufc_fights_cleaned['fighter_reach'].apply(float)-ufc_fights_cleaned['opponent_reach'].apply(float)
#ok so we keep ages absolute, but make differences for height and reach
ufc_best_df['fighter_age']=ufc_fights_cleaned['fighter_age']
ufc_best_df['opponent_age']=ufc_fights_cleaned['opponent_age']
#for some reason ages are stored as strings. Better if they are integers
ufc_best_df['fighter_age'] = ufc_best_df['fighter_age'].astype(int)
ufc_best_df['opponent_age'] = ufc_best_df['opponent_age'].astype(int)
#height and reach are best as differences
ufc_best_df['height_diff']=physical_stats_diff_df['height_diff']
ufc_best_df['reach_diff']=physical_stats_diff_df['reach_diff']

#This includes all the best choices in terms of taking differences
for col in punch_statistics:
    ufc_best_df[col]=ufc_fights_cleaned[col]

#adding stance to ufc_best_df
ufc_best_df['fighter_stance']=ufc_fights_cleaned['fighter_stance']
ufc_best_df['opponent_stance']=ufc_fights_cleaned['opponent_stance']




In [9]:
#best_df seems to be better than ufc_fights_cleaned at least
model=LogisticRegression()
X=ufc_best_df[ufc_best_df.columns[5:]]
y=ufc_best_df['result']
cross_val_score(model,X,y,cv=8).mean()

0.61354136034956575

In [12]:
model=LogisticRegression()
X=ufc_fights_cleaned[ufc_fights_cleaned.columns[6:]]
y=ufc_fights_cleaned['result']
cross_val_score(model,X,y,cv=8).mean()

0.61188634438256506

In [72]:
base_stats=[
    ['fighter_inf_total_strikes_landed_avg',
        'opponent_inf_total_strikes_landed_avg',
        'fighter_abs_total_strikes_landed_avg',
        'opponent_abs_total_strikes_landed_avg'],
    ['fighter_inf_total_strikes_attempts_avg',
'fighter_abs_total_strikes_attempts_avg',
'opponent_inf_total_strikes_attempts_avg',
'opponent_abs_total_strikes_attempts_avg'],
    ['fighter_inf_ground_strikes_attempts_avg',
        'fighter_abs_ground_strikes_attempts_avg',
        'opponent_inf_ground_strikes_attempts_avg',
        'opponent_abs_ground_strikes_attempts_avg',
        'fighter_inf_ground_strikes_landed_avg',
        'fighter_abs_ground_strikes_landed_avg',
        'opponent_inf_ground_strikes_landed_avg',
        'opponent_abs_ground_strikes_landed_avg'],
    ['fighter_inf_takedowns_landed_avg',
        'fighter_abs_takedowns_landed_avg',
        'opponent_inf_takedowns_landed_avg',
        'opponent_abs_takedowns_landed_avg',
     'fighter_inf_takedowns_attempts_avg',
'fighter_abs_takedowns_attempts_avg',
'opponent_inf_takedowns_attempts_avg',
'opponent_abs_takedowns_attempts_avg'],
    ['fighter_age',
        'opponent_age'],
    ['fighter_inf_leg_strikes_landed_avg',
        'fighter_inf_leg_strikes_attempts_avg',
        'fighter_abs_leg_strikes_landed_avg',
        'fighter_abs_leg_strikes_attempts_avg',
        'opponent_inf_leg_strikes_landed_avg',
        'opponent_inf_leg_strikes_attempts_avg',
        'opponent_abs_leg_strikes_landed_avg',
        'opponent_abs_leg_strikes_attempts_avg'],
        ['fighter_inf_clinch_strikes_attempts_avg',
        'fighter_abs_clinch_strikes_attempts_avg',
        'opponent_inf_clinch_strikes_attempts_avg',
        'opponent_abs_clinch_strikes_attempts_avg'],
         ['fighter_inf_clinch_strikes_landed_avg',
'fighter_abs_clinch_strikes_landed_avg',
'opponent_inf_clinch_strikes_landed_avg',
'opponent_abs_clinch_strikes_landed_avg'], 
['fighter_L5Y_wins_diff_2',
'fighter_L5Y_losses_diff_2',
'fighter_L2Y_wins_diff_2',
'fighter_L2Y_losses_diff_2'],
['fighter_inf_pass_avg',
        'fighter_abs_pass_avg',
        'opponent_inf_pass_avg',
        'opponent_abs_pass_avg'],
     ['fighter_inf_head_strikes_landed_avg',
        'fighter_abs_head_strikes_landed_avg',
        'opponent_inf_head_strikes_landed_avg',
        'opponent_abs_head_strikes_landed_avg'],
['fighter_ko_wins_diff_2'],
        ['fighter_sub_wins_diff_2'],
        ['fighter_ko_losses_diff_2'],
        ['fighter_inf_body_strikes_landed_avg',
        'fighter_inf_body_strikes_attempts_avg',
        'fighter_abs_body_strikes_landed_avg',
        'fighter_abs_body_strikes_attempts_avg',
        'opponent_inf_body_strikes_landed_avg',
        'opponent_inf_body_strikes_attempts_avg',
        'opponent_abs_body_strikes_landed_avg',
        'opponent_abs_body_strikes_attempts_avg'],
        ['fighter_inf_knockdowns_avg',
        'fighter_abs_knockdowns_avg',
        'opponent_inf_knockdowns_avg',
        'opponent_abs_knockdowns_avg'],
        ['fighter_stance',
        'opponent_stance',
        'height_diff',
        'reach_diff'],
['fighter_wins_diff_2',
'fighter_losses_diff_2'],
['fighter_L5Y_ko_wins_diff_2',
'fighter_L5Y_ko_losses_diff_2',
'fighter_L2Y_ko_wins_diff_2',
'fighter_L2Y_ko_losses_diff_2'],
['fighter_sub_losses_diff_2',
'fighter_L5Y_sub_wins_diff_2',
'fighter_L5Y_sub_losses_diff_2',
'fighter_L2Y_sub_wins_diff_2',
'fighter_L2Y_sub_losses_diff_2'],
['fighter_inf_reversals_avg',
'fighter_abs_reversals_avg',
'opponent_inf_reversals_avg',
'opponent_abs_reversals_avg'],
['fighter_inf_sub_attempts_avg',
'fighter_abs_sub_attempts_avg',
'opponent_inf_sub_attempts_avg',
'opponent_abs_sub_attempts_avg'],
['fighter_inf_sig_strikes_landed_avg',
'fighter_abs_sig_strikes_landed_avg',
'opponent_inf_sig_strikes_landed_avg',
'opponent_abs_sig_strikes_landed_avg'],
['fighter_inf_sig_strikes_attempts_avg',
'fighter_abs_sig_strikes_attempts_avg',
'opponent_inf_sig_strikes_attempts_avg',
'opponent_abs_sig_strikes_attempts_avg'],
['fighter_inf_head_strikes_attempts_avg',
'fighter_abs_head_strikes_attempts_avg',
'opponent_inf_head_strikes_attempts_avg',
'opponent_abs_head_strikes_attempts_avg'],
['fighter_inf_distance_strikes_landed_avg',
'fighter_abs_distance_strikes_landed_avg',
'opponent_inf_distance_strikes_landed_avg',
'opponent_abs_distance_strikes_landed_avg'],
['fighter_inf_distance_strikes_attempts_avg',
'fighter_abs_distance_strikes_attempts_avg',
'opponent_inf_distance_strikes_attempts_avg',
'opponent_abs_distance_strikes_attempts_avg']]

In [73]:
len(base_stats)

27

In [74]:
data=[]
for i in range(len(base_stats)):
    model=LogisticRegression()
    X=ufc_best_df[base_stats[i]]
    y=ufc_best_df['result']
    data.append([i,cross_val_score(model,X,y,cv=5).mean()])
    print([i,cross_val_score(model,X,y,cv=5).mean()])

[0, 0.56298730364764205]
[1, 0.54890613795369581]
[2, 0.54558396765029593]
[3, 0.54227247658580569]
[4, 0.56904364791080542]
[5, 0.5138075808156306]
[6, 0.53205528392815116]
[7, 0.5281817715657886]
[8, 0.56050224095902357]
[9, 0.53508633535413574]
[10, 0.56436774652723154]
[11, 0.51629987876239092]
[12, 0.51850715633907218]
[13, 0.53314500265742504]
[14, 0.5273496067353407]
[15, 0.51380682087373819]
[16, 0.53537783736452993]
[17, 0.55359920654055939]
[18, 0.53617948225401291]
[19, 0.53010860950396399]
[20, 0.51684931464255723]
[21, 0.52016576008550142]
[22, 0.55939881060134755]
[23, 0.55028124753789631]
[24, 0.54420465941349039]
[25, 0.55276132949778756]
[26, 0.53011471960542045]


In [75]:
[point[0] for point in data if point[1]>.55]

[0, 4, 8, 10, 17, 22, 23, 25]

In [76]:
starters=[]
for i in [0, 4, 8, 10, 17, 22, 23, 25]:
    starters.append(base_stats[i])

In [77]:
data_points=[]
SUB=subsets(len(starters))
for S in [R for R in SUB if len(R)>0]:
    candidate=[]
    for i in S:
        candidate.extend(starters[i-1])
    model=LogisticRegression()
    X=ufc_best_df[candidate]
    y=ufc_best_df['result']
    point=[S,cross_val_score(model,X,y,cv=5).mean()]
    data_points.append(point)
    print(point)

[(1,), 0.56298730364764205]
[(2,), 0.56904364791080542]
[(3,), 0.56050224095902357]
[(4,), 0.56436774652723154]
[(5,), 0.55359920654055939]
[(6,), 0.55939881060134755]
[(7,), 0.55028124753789631]
[(8,), 0.55276132949778756]
[(1, 2), 0.58258377397578465]
[(1, 3), 0.58066686271270274]
[(1, 4), 0.57072057774556018]
[(1, 5), 0.57072668573899477]
[(1, 6), 0.56547423562467414]
[(1, 7), 0.56023477619526818]
[(1, 8), 0.56934432192445539]
[(2, 3), 0.58701358804561865]
[(2, 4), 0.5792677021797209]
[(2, 5), 0.58149101126289748]
[(2, 6), 0.57595279137671473]
[(2, 7), 0.57568073639528783]
[(2, 8), 0.57374664528078456]
[(3, 4), 0.57956265027890452]
[(3, 5), 0.56050453132481004]
[(3, 6), 0.57680213078852938]
[(3, 7), 0.57044891749123261]
[(3, 8), 0.57404541838569267]
[(4, 5), 0.56464856297465071]
[(4, 6), 0.55552796962972861]
[(4, 7), 0.56325972384386203]
[(4, 8), 0.55745936721925826]
[(5, 6), 0.57044547561848691]
[(5, 7), 0.56492633333043485]
[(5, 8), 0.56962017766935524]
[(6, 7), 0.5596750473712044

[(1, 3, 4, 5, 6, 7), 0.58122125086330079]
[(1, 3, 4, 5, 6, 8), 0.58564190452399945]
[(1, 3, 4, 5, 7, 8), 0.58315418941683417]
[(1, 3, 4, 6, 7, 8), 0.5781821947511836]
[(1, 3, 5, 6, 7, 8), 0.58702388731358179]
[(1, 4, 5, 6, 7, 8), 0.57431172953446707]
[(2, 3, 4, 5, 6, 7), 0.595853378645264]
[(2, 3, 4, 5, 6, 8), 0.59613343725881263]
[(2, 3, 4, 5, 7, 8), 0.59557331476166042]
[(2, 3, 4, 6, 7, 8), 0.59253541015650213]
[(2, 3, 5, 6, 7, 8), 0.59502311577756895]
[(2, 4, 5, 6, 7, 8), 0.58232203091045209]
[(3, 4, 5, 6, 7, 8), 0.58426107851143094]
[(1, 2, 3, 4, 5, 6, 7), 0.60109817664011356]
[(1, 2, 3, 4, 5, 6, 8), 0.60165677082139335]
[(1, 2, 3, 4, 5, 7, 8), 0.59640356708924602]
[(1, 2, 3, 4, 6, 7, 8), 0.59695529333519892]
[(1, 2, 3, 5, 6, 7, 8), 0.6044130958752072]
[(1, 2, 4, 5, 6, 7, 8), 0.58922581367668792]
[(1, 3, 4, 5, 6, 7, 8), 0.58149711293226636]
[(2, 3, 4, 5, 6, 7, 8), 0.59557828073423458]
[(1, 2, 3, 4, 5, 6, 7, 8), 0.60109894185206059]


In [78]:
M=max([point[1] for point in data_points])
M

0.60552417624433241

In [81]:
S=[point[0] for point in data_points if point[1]==M]
S[0]

(1, 2, 3, 5, 6)

In [82]:
ground_zero=[]
for i in (1, 2, 3, 5, 6):
    ground_zero.append(starters[i-1])
ground_zero_joined=[]
for stats in ground_zero:
    ground_zero_joined.extend(stats)

In [87]:
X=ufc_best_df[ground_zero_joined]
y=ufc_best_df['result']
cross_val_score(model,X,y,cv=5).mean()

0.60552417624433241

In [84]:
add_backs=[]
for stats in base_stats:
    if stats not in ground_zero:
        add_backs.append(stats)

In [85]:
len(ground_zero)+len(add_backs)

27

In [98]:
#adding back two stats at a time
first_add_back_results=[]
for i in range(len(add_backs)):
    for j in range(i+1,len(add_backs)):
        candidate=[]
        candidate.extend(ground_zero_joined)
        candidate.extend(add_backs[i])
        candidate.extend(add_backs[j])
        X=ufc_best_df[candidate]
        y=ufc_best_df['result']
        first_add_back_results.append([(i,j),cross_val_score(model,X,y,cv=5).mean()])
        print([(i,j),cross_val_score(model,X,y,cv=5).mean()])

[(0, 1), 0.59972459537178524]
[(0, 2), 0.60800159208353466]
[(0, 3), 0.59778669716974353]
[(0, 4), 0.59667943442826632]
[(0, 5), 0.60082613536082885]
[(0, 6), 0.59944643977500034]
[(0, 7), 0.60165257006076645]
[(0, 8), 0.59833727190873742]
[(0, 9), 0.59861236981976684]
[(0, 10), 0.60220696453543021]
[(0, 11), 0.603036463245189]
[(0, 12), 0.60220848968926977]
[(0, 13), 0.5994460587500432]
[(0, 14), 0.60054759557705384]
[(0, 15), 0.59861351289463827]
[(0, 16), 0.59695643430204848]
[(0, 17), 0.59446566572517123]
[(0, 18), 0.59695300086139047]
[(0, 19), 0.60496710932491393]
[(0, 20), 0.59999434628528348]
[(0, 21), 0.59778554671679551]
[(1, 2), 0.60800731061992441]
[(1, 3), 0.60359391540560847]
[(1, 4), 0.59861847359715759]
[(1, 5), 0.60138052351142701]
[(1, 6), 0.59640928984167974]
[(1, 7), 0.60055179844570283]
[(1, 8), 0.59889471669108019]
[(1, 9), 0.59806370020555844]
[(1, 10), 0.60137937832853361]
[(1, 11), 0.6035931459776176]
[(1, 12), 0.59917248704686421]
[(1, 13), 0.61022489273462344

In [99]:
M1=max([point[1] for point in first_add_back_results])
M1

0.61464020782987849

In [100]:
[point for point in first_add_back_results if point[1]==M1]

[[(2, 13), 0.61464020782987849]]

In [101]:
ground_one=[]
ground_one.extend(ground_zero)
ground_one.append(add_backs[2])
ground_one.append(add_backs[13])
ground_one_joined=[]
for stats in ground_one:
    ground_one_joined.extend(stats)

In [102]:
X=ufc_best_df[ground_one_joined]
y=ufc_best_df['result']
cross_val_score(model,X,y,cv=5).mean()

0.61464020782987849

In [104]:
add_backs_one=[]
for stat in add_backs:
    if stat not in [add_backs[2],add_backs[13]]:
        add_backs_one.append(stat)

In [105]:
len(add_backs)-len(add_backs_one)

2

In [106]:
#again adding back two stats at a time
second_add_back_results=[]
for i in range(len(add_backs_one)):
    for j in range(i+1,len(add_backs_one)):
        candidate=[]
        candidate.extend(ground_one_joined)
        candidate.extend(add_backs_one[i])
        candidate.extend(add_backs_one[j])
        X=ufc_best_df[candidate]
        y=ufc_best_df['result']
        second_add_back_results.append([(i,j),cross_val_score(model,X,y,cv=5).mean()])
        print([(i,j),cross_val_score(model,X,y,cv=5).mean()])

[(0, 1), 0.61298542065708628]
[(0, 2), 0.61215363895961739]
[(0, 3), 0.61436625615575324]
[(0, 4), 0.61187929993646961]
[(0, 5), 0.60663182738884336]
[(0, 6), 0.60746131872052556]
[(0, 7), 0.6110490476088839]
[(0, 8), 0.61243179350239152]
[(0, 9), 0.61464211611669683]
[(0, 10), 0.61409306020747678]
[(0, 11), 0.6093896818125405]
[(0, 12), 0.61464211611669683]
[(0, 13), 0.60994636349095799]
[(0, 14), 0.60911649113431876]
[(0, 15), 0.61188006620242752]
[(0, 16), 0.60855828114002875]
[(0, 17), 0.60911533962735964]
[(0, 18), 0.61463792273414664]
[(0, 19), 0.61298046417061058]
[(1, 2), 0.6135352396702316]
[(1, 3), 0.60663106112288534]
[(1, 4), 0.61077432756077887]
[(1, 5), 0.60912030349191193]
[(1, 6), 0.61464250135769782]
[(1, 7), 0.61243407649010151]
[(1, 8), 0.60967087928491681]
[(1, 9), 0.60911991930492193]
[(1, 10), 0.61325784928539373]
[(1, 11), 0.61326547927063502]
[(1, 12), 0.61160496407535436]
[(1, 13), 0.61132681691065716]
[(1, 14), 0.61602638611003291]
[(1, 15), 0.6116064944992485

In [107]:
M2=max([point[1] for point in second_add_back_results])
M2

0.61602638611003291

In [108]:
[point for point in second_add_back_results if point[1]==M2]

[[(1, 14), 0.61602638611003291]]

In [109]:
ground_two=[]
ground_two.extend(ground_one)
ground_two.append(add_backs_one[1])
ground_two.append(add_backs_one[14])
ground_two_joined=[]
for stats in ground_two:
    ground_two_joined.extend(stats)

In [119]:
X=ufc_best_df[ground_two_joined]
y=ufc_best_df['result']
cross_val_score(model,X,y,cv=5).mean()

0.61602638611003291

In [120]:
add_backs_two=[]
for stat in add_backs_one:
    if stat not in [add_backs_one[1],add_backs_one[14]]:
        add_backs_two.append(stat)

In [121]:
len(add_backs_two)-len(add_backs_one)

-2

In [124]:
#again adding back two stats at a time
third_add_back_results=[]
for i in range(len(add_backs_two)):
    for j in range(i+1,len(add_backs_two)):
        candidate=[]
        candidate.extend(ground_two_joined)
        candidate.extend(add_backs_two[i])
        candidate.extend(add_backs_two[j])
        X=ufc_best_df[candidate]
        y=ufc_best_df['result']
        third_add_back_results.append([(i,j),cross_val_score(model,X,y,cv=5).mean()])
        print([(i,j),cross_val_score(model,X,y,cv=5).mean()])

[(0, 1), 0.61187930520652434]
[(0, 2), 0.60938815349666808]
[(0, 3), 0.60745559386007009]
[(0, 4), 0.60884100798228835]
[(0, 5), 0.61160115382578306]
[(0, 6), 0.61270841551324939]
[(0, 7), 0.61160267792561163]
[(0, 8), 0.61215897857907209]
[(0, 9), 0.61491721402772626]
[(0, 10), 0.60690654322090465]
[(0, 11), 0.61077509277272601]
[(0, 12), 0.60856361865146158]
[(0, 13), 0.61270803448829225]
[(0, 14), 0.60939234898724037]
[(0, 15), 0.6132605185681157]
[(0, 16), 0.60773107385006742]
[(0, 17), 0.61104828345094775]
[(1, 2), 0.60911610694732876]
[(1, 3), 0.61022146245599829]
[(1, 4), 0.60939311630720938]
[(1, 5), 0.60939349944018839]
[(1, 6), 0.61381110068518618]
[(1, 7), 0.61160115382578306]
[(1, 8), 0.61104980965879829]
[(1, 9), 0.60856400389246246]
[(1, 10), 0.6110509537876806]
[(1, 11), 0.61381148171014321]
[(1, 12), 0.60745750320089953]
[(1, 13), 0.61326014281321317]
[(1, 14), 0.61215593037941496]
[(1, 15), 0.61188083141437466]
[(1, 16), 0.61049808762888902]
[(1, 17), 0.609392350041251

In [125]:
M3=max([point[1] for point in third_add_back_results])
M3

0.61685511749982269

In [126]:
[point for point in third_add_back_results if point[1]==M3]

[[(5, 8), 0.61685511749982269]]

In [127]:
ground_three=[]
ground_three.extend(ground_two)
ground_three.append(add_backs_two[5])
ground_three.append(add_backs_two[8])
ground_three_joined=[]
for stats in ground_three:
    ground_three_joined.extend(stats)

In [133]:
X=ufc_best_df[ground_three_joined]
y=ufc_best_df['result']
cross_val_score(model,X,y,cv=7).mean()

0.61766805641832145

In [ ]:
#I have completely leveled out. Time to look at subsets...

In [134]:
len(ground_three)

11

In [135]:
data_points=[]
SUB=subsets(len(ground_three))
for S in [R for R in SUB if len(R)>0]:
    candidate=[]
    for i in S:
        candidate.extend(ground_three[i-1])
    model=LogisticRegression()
    X=ufc_best_df[candidate]
    y=ufc_best_df['result']
    point=[S,cross_val_score(model,X,y,cv=5).mean()]
    data_points.append(point)
    print(point)

[(1,), 0.56298730364764205]
[(2,), 0.56904364791080542]
[(3,), 0.56050224095902357]
[(4,), 0.55359920654055939]
[(5,), 0.55939881060134755]
[(6,), 0.54227247658580569]
[(7,), 0.53537783736452993]
[(8,), 0.54558396765029593]
[(9,), 0.51684931464255723]
[(10,), 0.56436774652723154]
[(11,), 0.53314500265742504]
[(1, 2), 0.58258377397578465]
[(1, 3), 0.58066686271270274]
[(1, 4), 0.57072668573899477]
[(1, 5), 0.56547423562467414]
[(1, 6), 0.57486347475261712]
[(1, 7), 0.56796120765412206]
[(1, 8), 0.55745823573850717]
[(1, 9), 0.562434813243753]
[(1, 10), 0.57072057774556018]
[(1, 11), 0.56933822341711937]
[(2, 3), 0.58701358804561865]
[(2, 4), 0.58149101126289748]
[(2, 5), 0.57595279137671473]
[(2, 6), 0.58176114425536374]
[(2, 7), 0.57320561408390502]
[(2, 8), 0.58231249580042499]
[(2, 9), 0.56683332213446702]
[(2, 10), 0.5792677021797209]
[(2, 11), 0.5673884765510232]
[(3, 4), 0.56050453132481004]
[(3, 5), 0.57680213078852938]
[(3, 6), 0.57376421985930537]
[(3, 7), 0.57459522896675042]


[(1, 2, 6, 7), 0.59834223366526773]
[(1, 2, 6, 8), 0.60082880991360532]
[(1, 2, 6, 9), 0.5917047694258829]
[(1, 2, 6, 10), 0.59253387130052038]
[(1, 2, 6, 11), 0.59640243139245119]
[(1, 2, 7, 8), 0.59116296563897974]
[(1, 2, 7, 9), 0.58452777806323897]
[(1, 2, 7, 10), 0.58618447141482777]
[(1, 2, 7, 11), 0.59281851486168613]
[(1, 2, 8, 9), 0.58756111353094442]
[(1, 2, 8, 10), 0.5850733963157575]
[(1, 2, 8, 11), 0.59004616146340982]
[(1, 2, 9, 10), 0.58258529702160222]
[(1, 2, 9, 11), 0.58645080048178833]
[(1, 2, 10, 11), 0.58037649639910338]
[(1, 3, 4, 5), 0.5848127931632634]
[(1, 3, 4, 6), 0.59365259008697435]
[(1, 3, 4, 7), 0.5889575984571388]
[(1, 3, 4, 8), 0.58204578676251839]
[(1, 3, 4, 9), 0.57956379967784166]
[(1, 3, 4, 10), 0.58149635299037405]
[(1, 3, 4, 11), 0.58039595923823462]
[(1, 3, 5, 6), 0.59420736031654053]
[(1, 3, 5, 7), 0.58757332424775899]
[(1, 3, 5, 8), 0.5900606530599124]
[(1, 3, 5, 9), 0.5809446235823883]
[(1, 3, 5, 10), 0.58066647325565801]
[(1, 3, 5, 11), 0.586

[(3, 6, 8, 11), 0.57928565462116599]
[(3, 6, 9, 10), 0.58453656693351586]
[(3, 6, 9, 11), 0.57625460108715965]
[(3, 6, 10, 11), 0.58011515016889215]
[(3, 7, 8, 9), 0.58233119131958744]
[(3, 7, 8, 10), 0.5842641309271317]
[(3, 7, 8, 11), 0.58398521222642163]
[(3, 7, 9, 10), 0.5839867363262502]
[(3, 7, 9, 11), 0.57983927228976229]
[(3, 7, 10, 11), 0.58674993563945654]
[(3, 8, 9, 10), 0.57376230735644307]
[(3, 8, 9, 11), 0.57293090668393143]
[(3, 8, 10, 11), 0.5756979151926982]
[(3, 9, 10, 11), 0.57127687839902064]
[(4, 5, 6, 7), 0.57652360786892953]
[(4, 5, 6, 8), 0.57293662943636503]
[(4, 5, 6, 9), 0.57569486699304118]
[(4, 5, 6, 10), 0.57486461255743371]
[(4, 5, 6, 11), 0.57017610256791318]
[(4, 5, 7, 8), 0.58150398824566996]
[(4, 5, 7, 9), 0.58177374126718995]
[(4, 5, 7, 10), 0.57873735654581648]
[(4, 5, 7, 11), 0.57791358585854602]
[(4, 5, 8, 9), 0.57459332173394295]
[(4, 5, 8, 10), 0.57238261071660368]
[(4, 5, 8, 11), 0.57708027268317186]
[(4, 5, 9, 10), 0.56906655051466037]
[(4, 5,

[(1, 3, 7, 8, 9), 0.58729479922013728]
[(1, 3, 7, 8, 10), 0.58453655112335168]
[(1, 3, 7, 8, 11), 0.59006180667489327]
[(1, 3, 7, 9, 10), 0.58729746534082639]
[(1, 3, 7, 9, 11), 0.59005951947113955]
[(1, 3, 7, 10, 11), 0.59033423108715699]
[(1, 3, 8, 9, 10), 0.57928602405200291]
[(1, 3, 8, 9, 11), 0.57790366656152903]
[(1, 3, 8, 10, 11), 0.57928525462401192]
[(1, 3, 9, 10, 11), 0.58176916053561678]
[(1, 4, 5, 6, 7), 0.58923460465498667]
[(1, 4, 5, 6, 8), 0.57984805588998445]
[(1, 4, 5, 6, 9), 0.57901283337378096]
[(1, 4, 5, 6, 10), 0.57790900512697274]
[(1, 4, 5, 6, 11), 0.57846302173871222]
[(1, 4, 5, 7, 8), 0.58260589450351741]
[(1, 4, 5, 7, 9), 0.58453311241263894]
[(1, 4, 5, 7, 10), 0.5756960026898359]
[(1, 4, 5, 7, 11), 0.58039556661915703]
[(1, 4, 5, 8, 9), 0.57016579592187355]
[(1, 4, 5, 8, 10), 0.57293090246788769]
[(1, 4, 5, 8, 11), 0.57514847615049902]
[(1, 4, 5, 9, 10), 0.57210293418202285]
[(1, 4, 5, 9, 11), 0.57679868048369598]
[(1, 4, 5, 10, 11), 0.57376649336091678]
[(1,

[(3, 4, 5, 6, 8), 0.58315725342665548]
[(3, 4, 5, 6, 9), 0.57956761730548956]
[(3, 4, 5, 6, 10), 0.58564420648390647]
[(3, 4, 5, 6, 11), 0.58399209280987985]
[(3, 4, 5, 7, 8), 0.58619440335997619]
[(3, 4, 5, 7, 9), 0.58951313811469597]
[(3, 4, 5, 7, 10), 0.58978747186778913]
[(3, 4, 5, 7, 11), 0.5953115759123716]
[(3, 4, 5, 8, 9), 0.58288215446161518]
[(3, 4, 5, 8, 10), 0.5815013189629481]
[(3, 4, 5, 8, 11), 0.58288558052419659]
[(3, 4, 5, 9, 10), 0.58177259924632951]
[(3, 4, 5, 9, 11), 0.57708065476213999]
[(3, 4, 5, 10, 11), 0.58039824960402131]
[(3, 4, 6, 7, 8), 0.58509210026700731]
[(3, 4, 6, 7, 9), 0.58564305813898032]
[(3, 4, 6, 7, 10), 0.59227290187922255]
[(3, 4, 6, 7, 11), 0.59255181320185601]
[(3, 4, 6, 8, 9), 0.58011666372861126]
[(3, 4, 6, 8, 10), 0.58453618274652597]
[(3, 4, 6, 8, 11), 0.57873698184492495]
[(3, 4, 6, 9, 10), 0.58039405727548188]
[(3, 4, 6, 9, 11), 0.57487147943874972]
[(3, 4, 6, 10, 11), 0.57901475114669809]
[(3, 4, 7, 8, 9), 0.58399093603286611]
[(3, 4, 7

...

[(2, 3, 4, 6, 7, 8), 0.60635520537798548]
[(2, 3, 4, 6, 7, 9), 0.60359543739741517]
[(2, 3, 4, 6, 7, 10), 0.60718011492408364]
[(2, 3, 4, 6, 7, 11), 0.60193530849714638]
[(2, 3, 4, 6, 8, 9), 0.5972365013476848]
[(2, 3, 4, 6, 8, 10), 0.60330965919346558]
[(2, 3, 4, 6, 8, 11), 0.60000046165679455]
[(2, 3, 4, 6, 9, 10), 0.59778326162106354]
[(2, 3, 4, 6, 9, 11), 0.59475298173109181]
[(2, 3, 4, 6, 10, 11), 0.59501930552799753]
[(2, 3, 4, 7, 8, 9), 0.5950345728765567]
[(2, 3, 4, 7, 8, 10), 0.60471071325711256]
[(2, 3, 4, 7, 8, 11), 0.59641922705688255]
[(2, 3, 4, 7, 9, 10), 0.59613457506362932]
[(2, 3, 4, 7, 9, 11), 0.59061773051944022]
[(2, 3, 4, 7, 10, 11), 0.59834796063374518]
[(2, 3, 4, 8, 9, 10), 0.60027556062183485]
[(2, 3, 4, 8, 9, 11), 0.59254189285082826]
[(2, 3, 4, 8, 10, 11), 0.60110543614050704]
[(2, 3, 4, 9, 10, 11), 0.5966786681623083]
[(2, 3, 5, 6, 7, 8), 0.60469697949448054]
[(2, 3, 5, 6, 7, 9), 0.60276480193685056]
[(2, 3, 5, 6, 7, 10), 0.6066325926007905]
[(2, 3, 5, 6, 7, 

[(5, 6, 8, 9, 10, 11), 0.56878497201732692]
[(5, 7, 8, 9, 10, 11), 0.56050874051752464]
[(6, 7, 8, 9, 10, 11), 0.57127497327423504]
[(1, 2, 3, 4, 5, 6, 7), 0.61464020782987849]
[(1, 2, 3, 4, 5, 6, 8), 0.60773106752600181]
[(1, 2, 3, 4, 5, 6, 9), 0.60110046805991102]
[(1, 2, 3, 4, 5, 6, 10), 0.60331232426014358]
[(1, 2, 3, 4, 5, 6, 11), 0.60441576726595092]
[(1, 2, 3, 4, 5, 7, 8), 0.61022489273462344]
[(1, 2, 3, 4, 5, 7, 9), 0.60524869098428025]
[(1, 2, 3, 4, 5, 7, 10), 0.60082689424871016]
[(1, 2, 3, 4, 5, 7, 11), 0.60552455305324571]
[(1, 2, 3, 4, 5, 8, 9), 0.59585451539606971]
[(1, 2, 3, 4, 5, 8, 10), 0.60055179844570283]
[(1, 2, 3, 4, 5, 8, 11), 0.60137937832853361]
[(1, 2, 3, 4, 5, 9, 10), 0.59834070429538444]
[(1, 2, 3, 4, 5, 9, 11), 0.60635442646389603]
[(1, 2, 3, 4, 5, 10, 11), 0.60221001695113108]
[(1, 2, 3, 4, 6, 7, 8), 0.61160649766128139]
[(1, 2, 3, 4, 6, 7, 9), 0.61188311967213938]
[(1, 2, 3, 4, 6, 7, 10), 0.60939387519509081]
[(1, 2, 3, 4, 6, 7, 11), 0.60911992562898765]
[

[(1, 3, 5, 8, 9, 10, 11), 0.58619477489883465]
[(1, 3, 6, 7, 8, 9, 10), 0.59420659932063724]
[(1, 3, 6, 7, 8, 9, 11), 0.59724032213736555]
[(1, 3, 6, 7, 8, 10, 11), 0.59530851401057228]
[(1, 3, 6, 7, 9, 10, 11), 0.59807286483073774]
[(1, 3, 6, 8, 9, 10, 11), 0.58950970256601598]
[(1, 3, 7, 8, 9, 10, 11), 0.59061161936397288]
[(1, 4, 5, 6, 7, 8, 9), 0.58896026668584978]
[(1, 4, 5, 6, 7, 8, 10), 0.58813077746218934]
[(1, 4, 5, 6, 7, 8, 11), 0.58896331488550679]
[(1, 4, 5, 6, 7, 9, 10), 0.58896064981882867]
[(1, 4, 5, 6, 7, 9, 11), 0.58785910666775232]
[(1, 4, 5, 6, 7, 10, 11), 0.59061925251124703]
[(1, 4, 5, 6, 8, 9, 10), 0.58122240237025991]
[(1, 4, 5, 6, 8, 9, 11), 0.58261048682921102]
[(1, 4, 5, 6, 8, 10, 11), 0.57929327722833068]
[(1, 4, 5, 6, 9, 10, 11), 0.57541823655009572]
[(1, 4, 5, 7, 8, 9, 10), 0.57818486403390534]
[(1, 4, 5, 7, 8, 9, 11), 0.58122887557848735]
[(1, 4, 5, 7, 8, 10, 11), 0.57321972360143947]
[(1, 4, 5, 7, 9, 10, 11), 0.57321362298608158]
[(1, 4, 5, 8, 9, 10, 11),

[(1, 2, 3, 4, 6, 7, 8, 10), 0.60800807793989342]
[(1, 2, 3, 4, 6, 7, 8, 11), 0.61160229900867635]
[(1, 2, 3, 4, 6, 7, 9, 10), 0.60718278420680538]
[(1, 2, 3, 4, 6, 7, 9, 11), 0.60608391877053847]
[(1, 2, 3, 4, 6, 7, 10, 11), 0.61160458410440821]
[(1, 2, 3, 4, 6, 8, 9, 10), 0.60496901866574337]
[(1, 2, 3, 4, 6, 8, 9, 11), 0.60635444122004933]
[(1, 2, 3, 4, 6, 8, 10, 11), 0.60855942316088929]
[(1, 2, 3, 4, 6, 9, 10, 11), 0.60192958152866893]
[(1, 2, 3, 4, 7, 8, 9, 10), 0.60165905802514719]
[(1, 2, 3, 4, 7, 8, 9, 11), 0.60359391118956462]
[(1, 2, 3, 4, 7, 8, 10, 11), 0.60193453801514452]
[(1, 2, 3, 4, 7, 9, 10, 11), 0.60055256260363898]
[(1, 2, 3, 4, 8, 9, 10, 11), 0.59889128535844405]
[(1, 2, 3, 5, 6, 7, 8, 9), 0.61188235551420322]
[(1, 2, 3, 5, 6, 7, 8, 10), 0.61132986827234692]
[(1, 2, 3, 5, 6, 7, 8, 11), 0.61050189787846043]
[(1, 2, 3, 5, 6, 7, 9, 10), 0.60857124969071363]
[(1, 2, 3, 5, 6, 7, 9, 11), 0.60994635927491425]
[(1, 2, 3, 5, 6, 7, 10, 11), 0.61326204477596602]
[(1, 2, 3, 5, 

[(1, 2, 3, 5, 6, 7, 8, 9, 10), 0.60912144762079423]
[(1, 2, 3, 5, 6, 7, 8, 9, 11), 0.61105629446114607]
[(1, 2, 3, 5, 6, 7, 8, 10, 11), 0.61491874234359867]
[(1, 2, 3, 5, 6, 7, 9, 10, 11), 0.60939730863574881]
[(1, 2, 3, 5, 6, 8, 9, 10, 11), 0.60800769586092529]
[(1, 2, 3, 5, 7, 8, 9, 10, 11), 0.60166058950305223]
[(1, 2, 3, 6, 7, 8, 9, 10, 11), 0.60745864100571612]
[(1, 2, 4, 5, 6, 7, 8, 9, 10), 0.59531119172538172]
[(1, 2, 4, 5, 6, 7, 8, 9, 11), 0.59669316819089824]
[(1, 2, 4, 5, 6, 7, 8, 10, 11), 0.59779966782845029]
[(1, 2, 4, 5, 6, 7, 9, 10, 11), 0.60055561185730688]
[(1, 2, 4, 5, 6, 8, 9, 10, 11), 0.59723688342665282]
[(1, 2, 4, 5, 7, 8, 9, 10, 11), 0.58673810278556648]
[(1, 2, 4, 6, 7, 8, 9, 10, 11), 0.60221536184064051]
[(1, 2, 5, 6, 7, 8, 9, 10, 11), 0.59558743271128245]
[(1, 3, 4, 5, 6, 7, 8, 9, 10), 0.59779967204449402]
[(1, 3, 4, 5, 6, 7, 8, 9, 11), 0.60055867375910643]
[(1, 3, 4, 5, 6, 7, 8, 10, 11), 0.59641616831711608]
[(1, 3, 4, 5, 6, 7, 9, 10, 11), 0.5911652570587772]


In [136]:
Msub1=max([point[1] for point in data_points])
Msub1

0.61685511749982269

In [137]:
[point[0] for point in data_points if point[1]==Msub1]

[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)]